In [1]:
from torchvision.datasets import VOCDetection
import matplotlib.pyplot as plt
from PIL import Image,ImageDraw

In [2]:
IMG_SIZE=448
S=7
C=20


# VOC数据集

In [ ]:
voc_ds=VOCDetection(root='./data',year='2012',image_set='train',download=True)

In [ ]:
img,label=voc_ds[4]
print(label)
img

In [ ]:
# 画框工具
draw=ImageDraw.Draw(img)

# 遍历每个object
for obj in label['annotation']['object']:
    box=obj['bndbox']
    draw.rectangle([int(box['xmin']),int(box['ymin']),int(box['xmax']),int(box['ymax'])],
                   outline='red',
                   width=3)
    draw.text([int(box['xmin'])+5,int(box['ymin'])+5],obj['name'],fill='red')

img

In [ ]:
img,label=voc_ds[4]

scaled_img=img.resize((IMG_SIZE,IMG_SIZE)) # 缩放至YoloV1的尺寸

# 缩放比
x_scale=IMG_SIZE/img.width
y_scale=IMG_SIZE/img.height

draw=ImageDraw.Draw(scaled_img)
for obj in label['annotation']['object']:
    box=obj['bndbox']
    draw.rectangle([int(box['xmin'])*x_scale,int(box['ymin'])*y_scale,int(box['xmax'])*x_scale,int(box['ymax'])*y_scale],
                   outline='red',
                   width=3)
    draw.text([int(box['xmin'])*x_scale+5,int(box['ymin'])*y_scale+5],obj['name'],fill='red')

scaled_img

# Dataset设计

In [7]:
# 引入dataset基类
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import torch

In [8]:
class YoloVOCDataset(Dataset):
    def __init__(self,voc_ds):
        super().__init__()
        self.voc_ds=voc_ds
        
        classdict=set()
        for _,label in self.voc_ds:
            for obj in label['annotation']['object']:
                classdict.add(obj['name'])
        self.id2name={i:c for i,c in enumerate(classdict)}
        self.name2id={c:i for i,c in self.id2name.items()}
    
    def __getitem__(self,index):
        img,label=self.voc_ds[index]
        
        scaled_img=img.resize((IMG_SIZE,IMG_SIZE)) 
    
        x_scale=IMG_SIZE/img.width
        y_scale=IMG_SIZE/img.height
        grid_size=IMG_SIZE//S
        
        x=ToTensor()(scaled_img)
        y=torch.zeros(S,S,10+C)
        
        for obj in label['annotation']['object']:
            box=obj['bndbox']
            classid=self.name2id[obj['name']]
            
            # normal coordinates
            xmin,ymin,xmax,ymax=int(box['xmin'])*x_scale,int(box['ymin'])*y_scale,int(box['xmax'])*x_scale,int(box['ymax'])*y_scale
            xcenter,ycenter=(xmin+xmax)/2,(ymin+ymax)/2
            width,height=xmax-xmin,ymax-ymin
            grid_i,grid_j=int(xcenter//grid_size),int(ycenter//grid_size)
            
            # yolo coordinates
            xcenter,ycenter=xcenter%grid_size/grid_size,ycenter%grid_size/grid_size
            width,height=width/IMG_SIZE,height/IMG_SIZE
            
            # targets
            y[grid_i,grid_j,0:5]=y[grid_i,grid_j,5:10]=torch.tensor([xcenter,ycenter,width,height,1])   # x,y,w,h,c
            y[grid_i,grid_j,10:]=torch.zeros(20)
            y[grid_i,grid_j,10+classid]=1
        return x,y
    
    def __len__(self):
        return len(self.voc_ds)

In [9]:
ds=YoloVOCDataset(voc_ds)

In [ ]:
x,y=ds[0]
x.shape,y.shape

# YoloV1 Model

In [11]:
from torch import nn 
import torch

device='cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
class YoloV1(nn.Module):
    def __init__(self):
        super().__init__()

        self.seq=nn.Sequential(
            # 448*448
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3), 
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2,stride=2),
            
            # 112*112
            nn.Conv2d(in_channels=64,out_channels=192,kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2,stride=2),
            
            # 56*56
            nn.Conv2d(in_channels=192,out_channels=128,kernel_size=1,stride=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=1,stride=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2,stride=2),
            
            # 28*28
            *[
                nn.Conv2d(in_channels=512,out_channels=256,kernel_size=1,stride=1),
                nn.LeakyReLU(0.1),
                nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,stride=1,padding=1),
                nn.LeakyReLU(0.1),
            ]*4,
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=1,stride=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=2,stride=2),
            
            # 14*14
            *[
                nn.Conv2d(in_channels=1024,out_channels=512,kernel_size=1,stride=1),
                nn.LeakyReLU(0.1),
                nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=3,stride=1,padding=1),
                nn.LeakyReLU(0.1),
            ]*2,
            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,stride=2,padding=1),
            nn.LeakyReLU(0.1),
            
            # 7*7
            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,stride=1,padding=1),
            nn.LeakyReLU(0.1),
            
            nn.Flatten(),
            nn.Linear(in_features=7*7*1024,out_features=4096),
            nn.Dropout(),
            nn.LeakyReLU(0.1),
            nn.Linear(in_features=4096,out_features=S*S*(10+C)),
            nn.Sigmoid(),
        )
        
    def forward(self,x): # x:(batch,3,448,448)
        y=self.seq(x)
        return torch.reshape(y,(-1,S,S,10+C)) # (batch,7,7,30)

In [ ]:
model=YoloV1().to(device)
x,y=ds[0]
outputs=model(x.unsqueeze(0).to(device))
outputs.shape

# Train

In [14]:
from torch.utils.data.dataloader import DataLoader
from torch import optim

In [15]:
def compute_iou(grid_row,grid_col,xywh_a,xywh_b):
    grid_size=IMG_SIZE//S
    
    # yolo coordinates
    xcenter_a,ycenter_a,w_a,h_a=xywh_a
    xcenter_b,ycenter_b,w_b,h_b=xywh_b
    
    # normal coordinates
    xcenter_a,ycenter_a=(grid_col+xcenter_a)*grid_size,(grid_row+ycenter_a)*grid_size
    w_a,h_a=w_a*IMG_SIZE,h_a*IMG_SIZE
    xcenter_b,ycenter_b=(grid_col+xcenter_b)*grid_size,(grid_row+ycenter_b)*grid_size
    w_b,h_b=w_b*IMG_SIZE,h_b*IMG_SIZE
    
    # border
    xmin_a,xmax_a,ymin_a,ymax_a=xcenter_a-w_a/2,xcenter_a+w_a/2,ycenter_a-h_a/2,ycenter_a+h_a/2
    xmin_b,xmax_b,ymin_b,ymax_b=xcenter_b-w_b/2,xcenter_b+w_b/2,ycenter_b-h_b/2,ycenter_b+h_b/2
    
    # IOU
    inter_xmin=max(xmin_a,xmin_b)
    inter_xmax=min(xmax_a,xmax_b)
    inter_ymin=max(ymin_a,ymin_b)
    inter_ymax=min(ymax_a,ymax_b)
    if inter_xmax<inter_xmin or inter_ymax<inter_ymin:
        return 0

    inter_area=(inter_xmax-inter_xmin)*(inter_ymax-inter_ymin) # 交集
    union_area=w_a*h_a+w_b*h_b-inter_area # 并集
    return inter_area/union_area # IOU

In [ ]:
lambda_coord=5
lambda_noojb=0.5

model=YoloV1().to(device)
dataloader=DataLoader(ds,batch_size=32,shuffle=True)
optimizer=optim.Adam(model.parameters(),lr=1e-4)

for epoch in range(50):
    for batch_x,batch_y in dataloader:
        batch_x,batch_y=batch_x.to(device),batch_y.to(device)
        batch_output=model(batch_x)
        
        loss=torch.tensor(0)
        for i in range(len(batch_x)):
            x=batch_x[i]
            y=batch_y[i]
            output=batch_output[i]
            # foreach grid 
            for row in range(S):    
                for col in range(S):
                    pred_grid=output[row,col] 
                    target_grid=y[row,col]
                    if not target_grid[4]>0: # no object in this grid
                        loss_c_noobj=(pred_grid[4])**2+(pred_grid[9])**2 # no object in grid,so target c is 0
                        loss=loss+lambda_noojb*loss_c_noobj
                        continue 
                    # IOU
                    iou_bbox1=compute_iou(row,col,pred_grid[:4],target_grid[:4])
                    iou_bbox2=compute_iou(row,col,pred_grid[5:9],target_grid[:4])
                    # 取IOU大的预测框的x,y,w,h,c
                    if iou_bbox1>iou_bbox2:
                        xywh=pred_grid[:4]
                        c_obj,c_noobj=pred_grid[4],pred_grid[9]
                    else:
                        xywh=pred_grid[5:9]
                        c_obj,c_noobj=pred_grid[9],pred_grid[4]
                    loss_xywh=(xywh[0]-target_grid[0])**2+(xywh[1]-target_grid[1])**2+(torch.sqrt(xywh[2])-torch.sqrt(target_grid[2]))**2+(torch.sqrt(xywh[3])-torch.sqrt(target_grid[3]))**2
                    loss_c_obj=(c_obj-1)**2
                    loss_c_noobj=(c_noobj)**2
                    loss_class=((pred_grid[10:]-target_grid[10:])**2).sum()
                    loss=loss+loss_xywh*lambda_coord+loss_c_obj+loss_c_noobj*lambda_noojb+loss_class
        loss=loss/len(batch_x)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.item())